<a href="https://colab.research.google.com/github/cagBRT/IntroToDNNwKeras/blob/master/DNNsimpleModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Clone the entire repo.
!git clone -l -s https://github.com/cagBRT/IntroToDNNwKeras.git cloned-repo
%cd cloned-repo

**Import libraries**

In [ ]:
# first neural network with keras tutorial
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import plot_model
import pandas as pd

**Get the data**<br>
The data is from a study on diabetes

**Relevant Information:**<br>

Several constraints were placed on the selection of these instances from a larger database.  In particular, all patients here are females at least 21 years old of Pima Indian heritage.  

Number of Instances: 768

Number of Attributes: 8 plus class 

For Each Attribute: (all numeric-valued)
   1. Number of times pregnant
   2. Plasma glucose concentration a 2 hours in an oral glucose tolerance test
   3. Diastolic blood pressure (mm Hg)
   4. Triceps skin fold thickness (mm)
   5. 2-Hour serum insulin (mu U/ml)
   6. Body mass index (weight in kg/(height in m)^2)
   7. Diabetes pedigree function
   8. Age (years)
   9. Class variable (0 or 1)


**Load the dataset**

In [ ]:
# load the dataset
dataset = loadtxt('pimaIndian.csv', delimiter=',')
# split into input (X) and output (y) variables
X = dataset[:,0:8]
y = dataset[:,8]

**Split the data into training and test sets**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20)

**Define the model with Keras**<br>
[Keras Sequential Model](https://keras.io/api/models/sequential/)

**Activation functions**<br>

The ReLU can be used with most types of neural networks.<br>

ReLu is also recommended as the default for both Multilayer Perceptron (MLP) and Convolutional Neural Networks (CNNs).<br>

Layer options: <br>
relu, sigmoid, tanh, elu, tf.keras.layers.LeakyReLU

In [ ]:
# define the keras model
model = Sequential()
#This adds the 8 inputs and the first hidden layer of 12 nodes
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# The patience parameter is the amount of epochs to check for improvement
early_stop = keras.callbacks.EarlyStopping(monitor='loss', patience=10)

**Train the model**

In [ ]:
# fit the keras model on the dataset
history=model.fit(X_train, y_train, epochs=150, 
                  batch_size=10,callbacks=[early_stop], validation_split=0.1)

In [ ]:
print(history.history['accuracy'])

In [ ]:
model.summary()

In [ ]:
keras.utils.plot_model(model, "model.png")

In [ ]:
# summarize the model
plot_model(model, 'model.png', show_shapes=True)


In [ ]:
history_dict = history.history
print(history_dict.keys())

In [ ]:
import matplotlib.pyplot as plt

axes = plt.gca()
axes.set_ylim([0,1])
plt.plot(history.history['val_accuracy'], label='testing accuracy')
plt.plot(history.history['accuracy'], label='training accuracy')
plt.title('Accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

**Evaluate the model**

In [ ]:
#model.evaluate returns the loss value & metrics values 
#for the model in test mode.
_,accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

**Assignment 1**<br>
Try changing the number of nodes in the hidden layers. <br>
How does the training and accuracy change?<br>

Try changing the number of epochs,the train/test ratio, and the activation layers. 



---

**Assignment 2**<br>
Use the California Housing dataset. <br>
Create and train a model to predict median_house_value. 

---



In [ ]:
house_df = pd.read_csv('/content/sample_data/california_housing_train.csv')

In [ ]:
print("shape:",house_df.shape)
house_df.head()

In [ ]:
house_value = house_df['median_house_value']
house_df.drop(columns=['median_house_value'],inplace=True)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    house_df,house_value, test_size=0.20)

In [ ]:
# define the keras model
#This adds the 8 inputs and the first hidden layer of 12 nodes
model_house= Sequential()
model_house.add(Dense(500, input_dim=8, activation='relu'))
model_house.add(Dense(100, activation='relu'))
model_house.add(Dense(50, activation='relu'))
model_house.add(Dense(1))


In [ ]:
# compile the keras model
model_house.compile(optimizer='adam', loss='mean_squared_error', metrics=["mean_squared_error"])

In [ ]:
# The patience parameter is the amount of epochs to check for improvement
early_stop = keras.callbacks.EarlyStopping( patience=10)

In [ ]:
# fit the keras model on the dataset
history=model_house.fit(X_train, y_train, epochs=50)

**Make predictions with the model**

In [ ]:
y_pred=model_house.predict(X_test)
y_pred[y_pred<0]=0
y_pred=y_pred.astype(int)

**Calculate the Root Mean Squared Error**

In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np
rms_dnn=np.sqrt(mean_squared_error(y_test, y_pred))
rms_dnn

In [ ]:
fig, ax=plt.subplots()
ax.scatter(y_test, y_pred, edgecolors=(0,0,0))
ax.plot([y_pred.min(), y_pred.max()],[y_pred.min(), y_pred.max()], 'k--', lw=4)
ax.set_xlabel("measured")
ax.set_ylabel('predicted')
plt.show()

**Compare the score for the sklearn model with the DNN model.**<br>
Use the sklearn linear regression model on the same training and test sets. <br>


In [ ]:
from sklearn.linear_model import LinearRegression
model_sklearn=LinearRegression().fit(X_train, y_train)
pred_sklearn=model_sklearn.predict(X_test)

**Score for SKLearn Linear Regression Model**<br>
Since this is errors - the lower the score the better

In [ ]:
print("Sklearn model performance:")
model_sklearn.score(X_test, y_test)
rms_sklearn=np.sqrt(mean_squared_error(y_test, pred_sklearn))
rms_sklearn

In [ ]:
print("DNN model performance:")
rms_dnn

In [ ]:
fig, ax=plt.subplots()
ax.scatter(y_test, y_pred, edgecolors=(0,0,0))
ax.plot([y_pred.min(), y_pred.max()],[y_pred.min(), y_pred.max()], 'g--', lw=2)
ax.plot([pred_sklearn.min(), pred_sklearn.max()], [pred_sklearn.min(), pred_sklearn.max()],color="red", linewidth=1)
ax.set_xlabel("measured")
ax.set_ylabel('predicted')
plt.show()